# importing necessary libraries

In [681]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from scipy import stats
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge

pd.set_option('display.max_columns', None) 

# Geting familiar with the data-set

In [682]:
train_ds = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
print(f"Full train dataset shape is {train_ds.shape}")
test_ds = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
print(f"Full test dataset shape is {test_ds.shape}")

Full train dataset shape is (1460, 81)
Full test dataset shape is (1459, 80)


# Performing Descriptive Statistical Analysis on Categorical Features.

In [683]:
train_ds.describe(include="object")
print(train_ds["GarageCars"].dtype)

int64


# Performing Descriptive Statistical Analysis on Numerical Features.

In [684]:
train_ds.describe(include=[int,float])

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


# Combining Train & Test Dataset for Easier Analysis

In [685]:
ids = test_ds.pop('Id')
target = train_ds[["SalePrice"]].reset_index(drop=True)
train_ds.drop(columns=["SalePrice"],inplace=True)
dataset = pd.concat([train_ds,test_ds]).reset_index(drop=True)

# drop ID column

In [686]:
dataset = dataset.drop('Id', axis=1)
print(f"Full train dataset shape after droping cools and rows{dataset.shape}")

Full train dataset shape after droping cools and rows(2919, 79)


# convert MSSubClass from int to str (categorical feature)

In [687]:
dataset["MSSubClass"] = dataset["MSSubClass"].astype(str)

# missing values Features with Missing Values More Than 45%

In [688]:
missing_values = dataset.isnull().sum()
features_with_missing_values = missing_values[missing_values > 0]
for feature_name, num_missing in features_with_missing_values.items():
    if (num_missing/dataset.shape[0]) * 100 > 45:
        print(f"Feature '{feature_name}' has {num_missing} missing values.")

Feature 'Alley' has 2721 missing values.
Feature 'FireplaceQu' has 1420 missing values.
Feature 'PoolQC' has 2909 missing values.
Feature 'Fence' has 2348 missing values.
Feature 'MiscFeature' has 2814 missing values.


# drop missing values Features with Missing Values More Than 45%

In [689]:
dataset = dataset.drop(["FireplaceQu","Fence","Alley","MiscFeature","PoolQC"], axis=1)

# Computing Total Missing Values and % of Misisng Values.

In [690]:
missing_values = dataset.isnull().sum()
features_with_missing_values = missing_values[missing_values > 0]
for feature_name, num_missing in features_with_missing_values.items():
    print(f"Feature {feature_name} has {num_missing} missing values. dtype = {dataset[feature_name].dtype}")

Feature MSZoning has 4 missing values. dtype = object
Feature LotFrontage has 486 missing values. dtype = float64
Feature Utilities has 2 missing values. dtype = object
Feature Exterior1st has 1 missing values. dtype = object
Feature Exterior2nd has 1 missing values. dtype = object
Feature MasVnrType has 24 missing values. dtype = object
Feature MasVnrArea has 23 missing values. dtype = float64
Feature BsmtQual has 81 missing values. dtype = object
Feature BsmtCond has 82 missing values. dtype = object
Feature BsmtExposure has 82 missing values. dtype = object
Feature BsmtFinType1 has 79 missing values. dtype = object
Feature BsmtFinSF1 has 1 missing values. dtype = float64
Feature BsmtFinType2 has 80 missing values. dtype = object
Feature BsmtFinSF2 has 1 missing values. dtype = float64
Feature BsmtUnfSF has 1 missing values. dtype = float64
Feature TotalBsmtSF has 1 missing values. dtype = float64
Feature Electrical has 1 missing values. dtype = object
Feature BsmtFullBath has 2 miss

# Filling  other Missing Values 

In [691]:
for col in ['GarageYrBlt', 'GarageArea', 'GarageCars']:
    dataset[col] = dataset[col].fillna(0)

for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    dataset[col] = dataset[col].fillna('None')

for col in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']:
    dataset[col] = dataset[col].fillna('None')
    

cat_cols = ['Utilities','Exterior1st','Exterior2nd','MasVnrType','Electrical',
            'KitchenQual','Functional','SaleType']
imputer = SimpleImputer(strategy="most_frequent")
dataset[cat_cols] = imputer.fit_transform(dataset[cat_cols])

dataset['MSZoning'] = dataset.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))
dataset["LotFrontage"] = dataset.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
dataset["MasVnrArea"]  = dataset.groupby("MasVnrType")["MasVnrArea"].transform(lambda x: x.fillna(x.median()))
dataset["BsmtFinSF1"]  = dataset.groupby("BsmtFinType1")["BsmtFinSF1"].transform(lambda x: x.fillna(x.median()))
dataset["BsmtFinSF2"]  = dataset.groupby("BsmtFinType2")["BsmtFinSF2"].transform(lambda x: x.fillna(x.median()))
dataset["BsmtFullBath"] = dataset["BsmtFullBath"].fillna(0.0)
dataset["BsmtHalfBath"] = dataset["BsmtHalfBath"].fillna(0.0)
dataset["TotalBsmtSF"]  = dataset["BsmtFinSF1"] + dataset["BsmtFinSF2"]
dataset["BsmtUnfSF"]    = dataset["BsmtUnfSF"].fillna(dataset["BsmtUnfSF"].median())


# Adding new features

In [692]:
dataset["SqFtPerRoom"] = dataset["GrLivArea"] / (dataset["TotRmsAbvGrd"] +
                                                       dataset["FullBath"] +
                                                       dataset["HalfBath"] +
                                                       dataset["KitchenAbvGr"])

dataset['Total_Home_Quality'] = dataset['OverallQual'] + dataset['OverallCond']

dataset['Total_Bathrooms'] = (dataset['FullBath'] + (0.5 * dataset['HalfBath']) +
                               dataset['BsmtFullBath'] + (0.5 * dataset['BsmtHalfBath']))

dataset["HighQualSF"] = dataset["1stFlrSF"] + dataset["2ndFlrSF"]

# handeling skewed data

In [693]:
skew_df = pd.DataFrame(dataset.select_dtypes(np.number).columns, columns=['Feature'])
skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: scipy.stats.skew(dataset[feature]))
skew_df['Absolute Skew'] = skew_df['Skew'].apply(abs)
skew_df['Skewed'] = skew_df['Absolute Skew'].apply(lambda x: True if x >= 0.5 else False)

for column in skew_df.query("Skewed == True")['Feature'].values:
    dataset[column] = np.log1p(dataset[column])
    


dataset_emcoded = pd.get_dummies(dataset)
scaler = StandardScaler()
scaler.fit(dataset_emcoded)

dataset_emcoded = pd.DataFrame(scaler.transform(dataset_emcoded), index=dataset_emcoded.index, columns=dataset_emcoded.columns)

# Cosine Transform for Cyclical Features

In [694]:
dataset['MoSold'] = (-np.cos(0.5236 * dataset['MoSold']))

# Ecode data for catboost and ridge Models

In [695]:
dataset_emcoded = pd.get_dummies(dataset)

# Scale the data for catboost and ridge Models

In [696]:
scaler = StandardScaler()
scaler.fit(dataset_emcoded)

dataset_emcoded = pd.DataFrame(scaler.transform(dataset_emcoded), index=dataset_emcoded.index, columns=dataset_emcoded.columns)

# Split data

In [697]:
train_df = pd.concat([dataset.iloc[:len(target["SalePrice"]),:],target],axis=1)
test_df = dataset.iloc[len(target["SalePrice"]):,:]
train_df_encoded = pd.concat([dataset_emcoded.iloc[:len(target["SalePrice"]),:],target],axis=1)
test_df_encoded = dataset_emcoded.iloc[len(target["SalePrice"]):,:]
train_df_encoded = train_df_encoded.drop("SalePrice", axis = 1)
print("Dimension of train data is:",train_df.shape)
print("Dimension of test data is:",test_df.shape)
dataset = train_df
target = dataset["SalePrice"]
dataset["SalePrice"] = np.log(dataset["SalePrice"])
dataset.head(5)

Dimension of train data is: (1460, 79)
Dimension of test data is: (1459, 78)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SqFtPerRoom,Total_Home_Quality,Total_Bathrooms,HighQualSF,SalePrice
0,60,RL,4.189655,9.042040,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,1.791759,7.602900,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.283204,Gd,TA,PConc,Gd,TA,No,GLQ,6.561031,Unf,0.0,5.017280,6.561031,GasA,Ex,Y,SBrkr,6.753438,6.751101,0.0,7.444833,0.693147,0.000000,2,0.693147,3,0.693147,Gd,2.197225,Typ,0.000000,Attchd,7.602900,RFn,2.0,548.0,TA,TA,Y,0.000000,4.127134,0.000000,0.0,0.0,0.0,0.0,-0.499998,2008,WD,Normal,4.966335,2.564949,3.5,7.444833,12.247694
1,20,RL,4.394449,9.169623,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,2.197225,7.589336,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000000,TA,TA,CBlock,Gd,TA,Gd,ALQ,6.886532,Unf,0.0,5.652489,6.886532,GasA,Ex,Y,SBrkr,7.141245,0.000000,0.0,7.141245,0.000000,0.693147,2,0.000000,3,0.693147,TA,1.945910,Typ,0.693147,Attchd,7.589336,RFn,2.0,460.0,TA,TA,Y,5.700444,0.000000,0.000000,0.0,0.0,0.0,0.0,0.866028,2007,WD,Normal,4.950335,2.708050,2.5,7.141245,12.109011
2,60,RL,4.234107,9.328212,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,1.791759,7.601902,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.093750,Gd,TA,PConc,Gd,TA,Mn,GLQ,6.188264,Unf,0.0,6.075346,6.188264,GasA,Ex,Y,SBrkr,6.825460,6.765039,0.0,7.488294,0.693147,0.000000,2,0.693147,3,0.693147,Gd,1.945910,Typ,0.693147,Attchd,7.601902,RFn,2.0,608.0,TA,TA,Y,0.000000,3.761200,0.000000,0.0,0.0,0.0,0.0,-0.000011,2008,WD,Normal,5.190732,2.564949,3.5,7.488294,12.317167
3,70,RL,4.110874,9.164401,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,1.791759,7.557995,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000000,TA,TA,BrkTil,TA,Gd,No,ALQ,5.379897,Unf,0.0,6.293419,5.379897,GasA,Gd,Y,SBrkr,6.869014,6.629363,0.0,7.448916,0.693147,0.000000,1,0.000000,3,0.693147,Gd,2.079442,Typ,0.693147,Detchd,7.600402,Unf,3.0,642.0,TA,TA,Y,0.000000,3.583519,5.609472,0.0,0.0,0.0,0.0,-0.499998,2006,WD,Abnorml,5.256337,2.564949,2.0,7.448916,11.849398
4,60,RL,4.442651,9.565284,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,1.791759,7.601402,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.860786,Gd,TA,PConc,Gd,TA,Av,GLQ,6.486161,Unf,0.0,6.196444,6.486161,GasA,Ex,Y,SBrkr,7.044033,6.960348,0.0,7.695758,0.693147,0.000000,2,0.693147,4,0.693147,Gd,2.302585,Typ,0.693147,Attchd,7.601402,RFn,3.0,836.0,TA,TA,Y,5.262690,4.442651,0.000000,0.0,0.0,0.0,0.0,-1.000000,2008,WD,Normal,5.136251,2.639057,3.5,7.695758,12.429216


# training set for catboost and ridge Models

In [698]:
train_df_encoded.head(5)

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SqFtPerRoom,Total_Home_Quality,Total_Bathrooms,HighQualSF,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Function

# test set for catboost and ridge Models

In [699]:
test_df_encoded.head(5)

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SqFtPerRoom,Total_Home_Quality,Total_Bathrooms,HighQualSF,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Function

# target variable for catboost and ridge Models

In [700]:
log_target = np.log(target)
log_target.head(5)

0    12.247694
1    12.109011
2    12.317167
3    11.849398
4    12.429216
Name: SalePrice, dtype: float64

# Split the dataset into training and testing datasets for Gradient Boosted TreesModel

In [701]:
def split_dataset(dataset, test_ratio=0.3):
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(dataset)

print(f"{len(train_ds_pd)} examples in training, {len(valid_ds_pd)} examples in  val testing.")

996 examples in training, 464 examples in  val testing.


# convert pandas dataframe into tensor

In [702]:
label = 'SalePrice'
whole_ds = tfdf.keras.pd_dataframe_to_tf_dataset(dataset, label=label, task = tfdf.keras.Task.REGRESSION)
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task = tfdf.keras.Task.REGRESSION)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label, task = tfdf.keras.Task.REGRESSION)

# TRAINING THE MODELS

In [703]:
kf = KFold(n_splits=10) 

results = {}

catboost_params = {
    'iterations': 6000,
    'learning_rate': 0.005,
    'depth': 4,
    'l2_leaf_reg': 1,
    'eval_metric':'RMSE',
    'early_stopping_rounds': 200,
    'random_seed': 42
}

ridge_params = {
    'alpha': 631.1412445239156
}

tuner = tfdf.tuner.RandomSearch(num_trials=60)
tuner.choice("min_examples", [2, 5, 7, 10])
local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8, 9,10,12 ,14,16,18,20])
global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])


GradientBoostedTreesModel= tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION, tuner = tuner)
catboost = CatBoostRegressor(**catboost_params, verbose=0)
ridgeModel = Ridge(**ridge_params)
    

GradientBoostedTreesModel.fit(whole_ds)
catboost.fit(train_df_encoded, log_target)
ridgeModel.fit(train_df_encoded, log_target)

GradientBoostedTreesModel.compile(
    optimizer='Adam',
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()])


GBT_train=GradientBoostedTreesModel.evaluate(train_ds)
GBT_test=GradientBoostedTreesModel.evaluate(valid_ds)

result_catboost = np.exp(np.sqrt(-cross_val_score(catboost, train_df_encoded,
                                         log_target, scoring='neg_mean_squared_error',
                                         cv=kf)))
result_catboost  = np.mean(result_catboost)


result_ridgeModel = np.exp(np.sqrt(-cross_val_score(ridgeModel, train_df_encoded,
                                         log_target, scoring='neg_mean_squared_error',
                                         cv=kf)))
results_ridgeModel = np.mean(result_ridgeModel)


print(f"ridgeModel rmse : {results_ridgeModel}")
print(f"catboost rmse : {result_catboost}")
print(f"GBT_train rmse : {np.exp(GBT_train[1])}")
print(f"GBT_test rmse : {np.exp(GBT_test[1])}")

Use /tmp/tmp8sq0yqta as temporary training directory
Reading training dataset...


[WARNING 23-07-21 00:58:27.4080 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-07-21 00:58:27.4080 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-07-21 00:58:27.4080 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Training dataset read in 0:00:01.090752. Found 1460 examples.
Training model...
Model trained in 0:00:58.953533
Compiling model...


[INFO 23-07-21 00:59:27.4466 UTC kernel.cc:1243] Loading model from path /tmp/tmp8sq0yqta/model/ with prefix 77e62678ed8e4555
[INFO 23-07-21 00:59:27.4610 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-07-21 00:59:27.4611 UTC kernel.cc:1075] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 373ms/step - loss: 0.0023 - root_mean_squared_error: 0.0480
ridgeModel rmse : 1.1324124964608804
catboost rmse : 1.1226817136586713
GBT_train rmse : 1.0513241054810696
GBT_test rmse : 1.0491902412644682


# variable importances in dicision forest

In [704]:
inspector = model.make_inspector()
inspector.evaluation()
ImportanceScore = inspector.variable_importances()["SUM_SCORE"]

last_17_elements = ImportanceScore
NumFeatures = [Feature[0][0] for Feature in last_17_elements]
print(NumFeatures)

['OverallQual', 'HighQualSF', 'Neighborhood', '1stFlrSF', 'Total_Bathrooms', 'KitchenQual', 'ExterQual', 'GarageCars', 'Total_Home_Quality', 'MSSubClass', 'GarageArea', 'GrLivArea', 'CentralAir', 'TotalBsmtSF', 'LotArea', 'BsmtFinSF1', 'YearBuilt', 'GarageType', 'Fireplaces', 'YearRemodAdd', 'SaleCondition', 'OverallCond', 'MSZoning', 'Exterior2nd', 'BsmtFinType1', 'GarageYrBlt', 'Exterior1st', 'BsmtUnfSF', 'SqFtPerRoom', 'BsmtQual', '2ndFlrSF', 'BsmtExposure', 'Functional', 'Condition1', 'TotRmsAbvGrd', 'HeatingQC', 'OpenPorchSF', 'BsmtCond', 'LotFrontage', 'ScreenPorch', 'WoodDeckSF', 'SaleType', 'MoSold', 'GarageQual', 'BsmtFinType2', 'LotConfig', 'PavedDrive', 'RoofStyle', 'LandContour', 'YrSold', 'BldgType', 'EnclosedPorch', 'BedroomAbvGr', 'Foundation', 'MasVnrArea', 'GarageCond', 'LotShape', 'ExterCond', 'GarageFinish', 'BsmtFinSF2', 'RoofMatl', 'MasVnrType', 'HouseStyle', 'HalfBath', 'Electrical', 'Condition2', 'BsmtFullBath', 'BsmtHalfBath', 'MiscVal', 'LandSlope']


# predicting

In [735]:
sample_submission_path = "../input/house-prices-advanced-regression-techniques/sample_submission.csv"
sample_submission = pd.read_csv(sample_submission_path)
ids = sample_submission.pop('Id')
test_d = tfdf.keras.pd_dataframe_to_tf_dataset(
    test_df,
    task = tfdf.keras.Task.REGRESSION)
preds = model.predict(test_d)
output = pd.DataFrame({'SalePrice': np.exp(preds).squeeze()})
print(np.asarray(output).flatten())
print(np.exp(np.exp(catboost.predict(test_df_encoded))))
prediction = 0.3 * np.asarray(output).flatten() + 0.4 * np.exp(catboost.predict(test_df_encoded)) + 0.3 * np.exp(ridgeModel.predict(test_df_encoded))

2/2 [==============================] - 0s 17ms/step
[118380.09 160926.61 187882.98 ... 159357.7  132131.45 209097.92]
[inf inf inf ... inf inf inf]


/tmp/ipykernel_32/919660219.py:10: RuntimeWarning: overflow encountered in exp
  print(np.exp(np.exp(catboost.predict(test_df_encoded))))


# submission

In [736]:
submission = pd.concat([ids, pd.Series(prediction, name='SalePrice')], axis=1)
submission.head(10)

,Id,SalePrice
0,1461,122105.572613
1,1462,157717.857519
2,1463,184128.076088
3,1464,192787.239825
4,1465,187519.393644
5,1466,172761.494355
6,1467,171309.794109
7,1468,165717.091489
8,1469,184552.704849
9,1470,123757.894245


In [728]:
submission.to_csv('./submission.csv', index=False, header=True)